In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [29]:
from PIL import Image, ImageDraw
import os
import math

# Function to generate a shape image with a specific color
def generate_shape_image(width, height, shape, color):
    image = Image.new("RGB", (width, height), "white")
    draw = ImageDraw.Draw(image)
    
    # Draw the shape with the corresponding color
    if shape == "rectangle":
        draw.rectangle([10, 20, width-10, height-10], fill=color)
    elif shape == "square":
        draw.rectangle([10, 10, width-10, height-10], fill=color)
    elif shape == "triangle":
        draw.polygon([(width//2, 10), (10, height-10), (width-10, height-10)], fill=color)
    elif shape == "pentagon":
        side_length = width // 3
        center = (width // 2, height // 2)
        points = []
        for i in range(5):
            angle = math.radians(360 / 5 * i - 90)
            x = center[0] + side_length * math.cos(angle)
            y = center[1] + side_length * math.sin(angle)
            points.append((x, y))
        draw.polygon(points, fill=color)
    elif shape == "trapezium":
        draw.polygon([(20, 10), (width-20, 10), (width-40, height-10), (40, height-10)], fill=color)
    
    return image

# Function to save images to a directory
def save_images(output_dir):
    os.makedirs(output_dir, exist_ok=True)
    
    # Define colors for each shape
    shape_colors = {
        "rectangle": "red",
        "square": "green",
        "triangle": "blue",
        "pentagon": "orange",
        "trapezium": "purple",
    }
    
    for i, (shape, color) in enumerate(shape_colors.items()):
        image = generate_shape_image(100, 100, shape, color)
        image_path = os.path.join(output_dir, f"image_{i}.png")
        image.save(image_path)

# Generate and save 6 images to a directory named "shapes_dataset"
save_images("/kaggle/working/shapes_dataset4")

In [30]:
from PIL import Image
import os

# Function to convert colored images to grayscale
def convert_to_grayscale(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)  # Create the output directory if it doesn't exist
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.png'):
            # Load the image
            image_path = os.path.join(input_dir, file_name)
            image = Image.open(image_path)
            
            # Convert to grayscale
            grayscale_image = image.convert('L')
            
            # Save the grayscale image
            output_path = os.path.join(output_dir, file_name)
            grayscale_image.save(output_path)

# Convert colored images in the 'colored_dataset' directory to grayscale and save them to a new directory
convert_to_grayscale('/kaggle/working/shapes_dataset4', '/kaggle/working/grayscale_dataset4')

In [31]:
import os
import cv2
import numpy as np

# Function to preprocess image before edge detection
def preprocess_image(image):
    # Apply thresholding to create binary image
    _, binary_image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
    
    # Apply morphological closing to fill gaps and smooth edges
    kernel = np.ones((5, 5), np.uint8)
    closed_image = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernel)
    
    return closed_image

# Function to perform edge detection on grayscale images
def apply_edge_detection(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)  # Create the output directory if it doesn't exist
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.png'):
            # Load the grayscale image
            image_path = os.path.join(input_dir, file_name)
            grayscale_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            
            # Preprocess image
            processed_image = preprocess_image(grayscale_image)
            
            # Apply edge detection using Canny
            edges = cv2.Canny(processed_image, 100, 200)  # Adjust the threshold values as needed
            
            # Save the edge-detected image
            output_path = os.path.join(output_dir, file_name)
            cv2.imwrite(output_path, edges)

# Apply edge detection to grayscale images in the 'grayscale_dataset' directory and save them to a new directory
apply_edge_detection('/kaggle/working/grayscale_dataset4', '/kaggle/working/edge_detected_dataset4')

In [32]:
import os
import cv2
import numpy as np

# Define input and output directories
input_dir = '/kaggle/working/shapes_dataset4'
output_dir = '/kaggle/working/rotated_dataset4'

# Define rotation angles
angles = [-45, -30, -15, 15, 30, 45]

# Function to rotate images and save them to the output directory
def rotate_images(input_dir, output_dir, angles):
    os.makedirs(output_dir, exist_ok=True)
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.png'):
            image_path = os.path.join(input_dir, file_name)
            image = cv2.imread(image_path)
            for angle in angles:
                rotated_image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
                output_path = os.path.join(output_dir, f"{os.path.splitext(file_name)[0]}_{angle}.png")
                cv2.imwrite(output_path, rotated_image)

# Rotate images in the input directory and save them to the output directory
rotate_images(input_dir, output_dir, angles)

In [14]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# Define input and output directories
input_dir = '/kaggle/working/rotated_dataset4'

# Load images and corresponding rotation angles
images = []
angles = []

for file_name in os.listdir(input_dir):
    if file_name.endswith('.png'):
        image_path = os.path.join(input_dir, file_name)
        image = tf.keras.preprocessing.image.load_img(image_path, target_size=(100, 100))
        image_array = tf.keras.preprocessing.image.img_to_array(image)
        images.append(image_array)
        angle = int(file_name.split('_')[-1].split('.')[0])
        angles.append(angle)

images = np.array(images)
angles = np.array(angles)

# Normalize images
images = images / 255.0

# Define the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

# Compile the model
model.compile(optimizer='adam',
              loss='mean_squared_error')

# Train the model
model.fit(images, angles, epochs=10, batch_size=32, validation_split=0.2)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 899.4020 - val_loss: 1899.4974
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step - loss: 870.4240 - val_loss: 2149.3838
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step - loss: 873.3550 - val_loss: 1934.5177
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 278ms/step - loss: 859.5173 - val_loss: 1893.6219
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step - loss: 859.8173 - val_loss: 2004.0947
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step - loss: 851.8667 - val_loss: 2274.7463
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step - loss: 855.4915 - val_loss: 2187.4717
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step - loss: 848.6492 - val_loss: 2087.1472
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step - loss: 848.1973 - val_loss: 2073.3955
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 296ms/step - loss: 849.1132 - val_loss: 2141.7854


In [35]:
import os
import cv2
import numpy as np

# Function to preprocess image before edge detection
def preprocess_image(image):
    # Apply thresholding to create binary image
    _, binary_image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
    
    # Apply morphological closing to fill gaps and smooth edges
    kernel = np.ones((5, 5), np.uint8)
    closed_image = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernel)
    
    return closed_image

# Function to perform edge detection on grayscale images
def apply_edge_detection(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)  # Create the output directory if it doesn't exist
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.png'):
            # Load the grayscale image
            image_path = os.path.join(input_dir, file_name)
            grayscale_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            
            # Preprocess image
            processed_image = preprocess_image(grayscale_image)
            
            # Apply edge detection using Canny
            edges = cv2.Canny(processed_image, 100, 200)  # Adjust the threshold values as needed
            
            # Save the edge-detected image
            output_path = os.path.join(output_dir, file_name)
            cv2.imwrite(output_path, edges)

# Apply edge detection to grayscale images in the 'grayscale_dataset' directory and save them to a new directory
apply_edge_detection('/kaggle/working/grayscale_dataset3', '/kaggle/working/edge_detected_dataset4')